In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from Bio.Seq import Seq
from Bio import motifs
import random

np.random.seed(1337)

%matplotlib inline

### Parameters for plotting model results ###
pd.set_option("display.max_colwidth",100)
sns.set(style="ticks", color_codes=True)
plt.rcParams['font.weight'] = 'normal'
plt.rcParams['axes.labelweight'] = 'normal'
plt.rcParams['axes.labelpad'] = 5
plt.rcParams['axes.linewidth']= 2
plt.rcParams['xtick.labelsize']= 14
plt.rcParams['ytick.labelsize']= 14
plt.rcParams['xtick.major.size'] = 6
plt.rcParams['ytick.major.size'] = 6
plt.rcParams['xtick.minor.size'] = 3
plt.rcParams['ytick.minor.size'] = 3
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.minor.width'] = 1
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['ytick.major.width'] = 2
plt.rcParams['xtick.color'] = 'black'
plt.rcParams['ytick.color'] = 'black'
plt.rcParams['axes.labelcolor'] = 'black'
plt.rcParams['axes.edgecolor'] = 'black'

## General Utility Functions For Data Processing

In [4]:
def test_data(df, model, test_seq, obs_col, output_col='pred'):
    '''Predict mean ribosome load using model and test set UTRs'''
    
    # Scale the test set mean ribosome load
    scaler = preprocessing.StandardScaler()
    scaler.fit(df[obs_col].values.reshape(-1,1))   #LAE: .values added, pd deprecation
    
    # Make predictions
    predictions = model.predict(test_seq).reshape(-1)
    
    # Inverse scaled predicted mean ribosome load and return in a column labeled 'pred'
    df.loc[:,output_col] = scaler.inverse_transform(predictions)
    return df


def one_hot_encode(df, col='utr', seq_len=50):
    # Dictionary returning one-hot encoding of nucleotides. 
    nuc_d = {'a':[1,0,0,0],'c':[0,1,0,0],'g':[0,0,1,0],'t':[0,0,0,1], 'n':[0,0,0,0]}
    
    # Creat empty matrix.
    vectors=np.empty([len(df),seq_len,4])
    
    # Iterate through UTRs and one-hot encode
    for i,seq in enumerate(df[col].str[:seq_len]): 
        seq = seq.lower()
        a = np.array([nuc_d[x] for x in seq])
        vectors[i] = a
    return vectors


def r2(x,y):
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
    return r_value**2

## Load Data and Create Train/Test Datasets

In [6]:
#df = pd.read_pickle('../data/egfp_unmod_1.pkl')  #LAE: do not have this pickle file...?
df = pd.read_csv('../data/egfp_unmod_1.csv')
df.sort_values('total_reads', inplace=True, ascending=False)
df.reset_index(inplace=True, drop=True)
df = df.iloc[:280000]

# The training set has 260k UTRs and the test set has 20k UTRs.
e_test = df.iloc[:20000]
e_train = df.iloc[20000:]

# One-hot encode both training and test UTRs
seq_e_train = one_hot_encode(e_train,seq_len=50)
seq_e_test = one_hot_encode(e_test, seq_len=50)

# Scale the training mean ribosome load values
# LAE: added .values before reshape due to pandas naming deprecation
e_train.loc[:,'scaled_rl'] = preprocessing.StandardScaler().fit_transform(e_train.loc[:,'rl'].values.reshape(-1,1))

label_e_train = e_train['scaled_rl']
label_e_test = e_test['rl']

## Create 2D Embedding for Sequences to Allow Use of Random Forest Model

In [11]:
# Most simplistic version: simply concatenate the one-hot encodings:
# ATG -> [ [1,0,0,0],[0,1,0,0],[0,0,0,1] ] -> [1,0,0,0,0,1,0,0,0,0,0,1]

flat_seq_e_train = np.array([ seq.flatten(order='C') for seq in seq_e_train])
flat_seq_e_test = np.array([ seq.flatten(order='C') for seq in seq_e_test])

## Create Model Structure (Random Forest) & Train Model

In [22]:
def train_model(train_data,train_labels,n_trees=100,warm_start=True):
    ''' Build model archicture and fit.'''
    model = RandomForestRegressor(n_estimators=n_trees,criterion='mse',warm_start=warm_start)
    model.fit(train_data,train_labels)
    return model

In [23]:
# Split data so it fits in memory (batch size was 128, going to 130 for even split)
batch_size = 130
train_data_sets = np.split(flat_seq_e_train,(len(flat_seq_e_train)/batch_size))
train_label_sets = np.split(label_e_train,(len(flat_seq_e_train)/batch_size))

In [29]:
np.random.seed(1337)
start_trees,n = 20,0
epochs = 1
for e in range(epochs):
    print "e:",e
    for i,(X,y) in enumerate(zip(train_data_sets,train_label_sets)[:100]):
        model = train_model(flat_seq_e_train,label_e_train,n_trees=start_trees+n,warm_start=True)
        if (i+1) % 10 == 0:
            n += 1
            print "\t i:",n*10 

e: 0


KeyboardInterrupt: 